In [ ]:
import sys
import os
import time
import numpy as np
import tensorflow as tf
import gpflow
import soundfile
import pickle
sys.path.append('../../../../')
import gpitch
from gpitch.amtgp import logistic
from gpitch import myplots
import soundfile
import sounddevice
import peakutils
from scipy import signal

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']= (16, 3)

In [ ]:
active_device = sys.argv[1]  # configure gpu usage and plotting
gpitch.amtgp.init_settings(visible_device = active_device, interactive=True)

In [ ]:
saveloc = '../../../../../results/files/svi/notebook/'
dirloc = '../../../../../datasets/maps/sample_rate_16khz/'
pattern = '*F*.wav'
bounds = [21, 109]
midi = np.asarray([str(i) for i in range(bounds[0], bounds[1])]).reshape(-1,)
filel = gpitch.amtgp.load_filenames(directory=dirloc, pattern=pattern, bounds=bounds)
Np = filel.size  # number of pitches to analyze

In [ ]:
fs = 16e3  # sample frequency
N = 16000  # number of data points to load
x = np.linspace(0, (N-1.)/fs, N).reshape(-1, 1)  # time vector
Nc = 10  #  number of harmonics for component kernels
dec = 160  # decimation factor
minibatch_size = 200
maxiter = 2000
learning_rate = 0.01

In [ ]:
Y = np.zeros((N, Np))
ideal_f0 = np.zeros((Np,1))
Fs = [None]*Np
Ss = [None]*Np 
F  = np.zeros((N//2, 1))
S  = np.zeros((N//2, Np))
thres = np.zeros((Np, 1))

In [ ]:
maxh = 20
for i in range(0, Np):  # load data
    y, fs = gpitch.amtgp.wavread(dirloc + filel[i], start=5000, N=N)  # load data       
    Y[:, i] = y.reshape(-1,).copy()
    ideal_f0[i] = gpitch.amtgp.midi2frec(int(midi[i]))
    
    Fs[i], Ss[i], F[:, 0], S[:, i], thres[i] = gpitch.amtgp.init_com_params(y=y, fs=fs, 
                                                                               maxh=maxh, 
                                                                               ideal_f0=ideal_f0[i],
                                                                               scaled=True, 
                                                                               win_size=10)

In [ ]:
plt.figure(1, figsize=(16, 80))  
plt.figure(2, figsize=(16, 80))
nrow, ncol = 22, 4
maxh = 20
for i in range(0, Np):
    y = Y[:, i].reshape(-1,1).copy()
    
    plt.figure(1)
    plt.subplot(nrow, ncol, i+1)
    plt.plot(x, y)
    plt.legend(['pitch ' + midi[i]])
         
    plt.figure()
    plt.plot(F, S[:,i]/ np.max(S[:,i]))
    plt.plot(Fs[i], Ss[i]/np.max(Ss[i]), 'xk', mew=2)
    plt.plot(F, 0.*S[:,i] + thres[i])
    plt.legend(['pitch ' + midi[i]])
    plt.xlim([0, 1.1*np.max(Fs[i])])

In [ ]:
plt.figure(),  plt.title('Number of harmonics in component kernel for each pitch')
for i in range(Np):
    plt.plot(i, Fs[i].size, '.k')

In [ ]:
for i in range(Np):
    y = Y[:, i].reshape(-1,1).copy()
    z = np.vstack((x[::dec].copy(), x[-1].copy()))
    
    kern_com = gpitch.kernels.MaternSpecMix(input_dim=1, lengthscales=0.1, variances=Ss[i],
                                            frequencies=Fs[i], Nc=Fs[i].size)
    kern_act = gpflow.kernels.Matern32(input_dim=1, lengthscales=0.25, variance=10.)

    m = gpitch.modgp.ModGP(x=x, y=y, z=z, kern_com=kern_com, kern_act=kern_act, whiten=True,
                           minibatch_size=minibatch_size)
    
    m.fixed_msmkern_params(freq=False, var=True)
    m.kern_com.lengthscales.fixed = False
    m.kern_com.lengthscales.transform = gpflow.transforms.Logistic(0., 10.0)
    m.kern_act.fixed = False
    m.likelihood.variance.fixed = False
    m.z.fixed = True
    print('learning from ' + filel[i])
    m.optimize_svi(maxiter=maxiter, learning_rate=learning_rate)
    pickle.dump(m, open(saveloc + "maps_pitch_" + midi[i] + ".p", "wb"))
    tf.reset_default_graph()